In [21]:
import tensorflow as tf
from tensorflow import  keras
from tensorflow.keras import layers
import numpy as np
import joblib
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [22]:
X_train, y_train = joblib.load('emotion_recognition_data_train.pkl')


In [23]:
X_test, y_test = joblib.load('emotion_recognition_data_test.pkl')

In [24]:
base_model  = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), 
    include_top=False, 
    weights="imagenet"
)

In [25]:
base_input = base_model.input
base_output = base_model.output

In [26]:
# Add custom layers on top of the MobileNetV2 base
x = layers.GlobalAveragePooling2D()(base_output)  # Global average pooling
x = layers.Dense(128, activation='relu')(x)       # Dense layer with 128 units                       # Dropout for regularization
final_output = layers.Dense(7, activation='softmax')(x)  # Output layer for 10 classes

In [27]:
model = keras.Model(inputs=base_input, outputs=final_output)
for layer in base_model.layers:
    layer.trainable = False
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,855 (9.24 MB)

 Trainable params: 164,871 (644.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [28]:
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.5,          # Reduce learning rate by a factor of 0.5
    patience=3,          # Wait for 3 epochs before reducing
    min_lr=1e-6,         # Minimum learning rate
    verbose=1
)

# Train the model
history = model.fit(
    X_train, 
    y_train, 
    epochs=20,  # Set the number of epochs
    batch_size=32,  # Set the batch size
    validation_data=(X_test, y_test),
    callbacks=[lr_scheduler]  # Use the validation set
)


Epoch 1/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 53s 230ms/step - accuracy: 0.2807 - loss: 1.8638 - val_accuracy: 0.4286 - val_loss: 1.5488 - learning_rate: 0.0010
Epoch 2/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 50s 246ms/step - accuracy: 0.4186 - loss: 1.5350 - val_accuracy: 0.5571 - val_loss: 1.4546 - learning_rate: 0.0010
Epoch 3/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 49s 244ms/step - accuracy: 0.4696 - loss: 1.4080 - val_accuracy: 0.5143 - val_loss: 1.4951 - learning_rate: 0.0010
Epoch 4/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 46s 227ms/step - accuracy: 0.4943 - loss: 1.3487 - val_accuracy: 0.4571 - val_loss: 1.4444 - learning_rate: 0.0010
Epoch 5/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 49s 242ms/step - accuracy: 0.5376 - loss: 1.2698 - val_accuracy: 0.5571 - val_loss: 1.3803 - learning_rate: 0.0010
Epoch 6/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 46s 229ms/step - accuracy: 0.5536 - loss: 1.2266 - val_accuracy: 0.5143 - val_loss: 1.4205 - learning_rate: 0.0010
Epoch 7/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 48s 238ms/step - accuracy: 0.5

In [29]:
from sklearn.metrics import accuracy_score

# Get model predictions on the test set
y_pred = model.predict(X_test)

# Convert the predictions from one-hot encoding to class indices
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate accuracy using sklearn's accuracy_score
accuracy = accuracy_score(y_true_classes, y_pred_classes)

print(f"Test Accuracy: {accuracy}")


2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/stepWARNING:tensorflow:6 out of the last 12 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000027C1E6C87C0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 958ms/step
Test Accuracy: 0.5142857142857142
